In [2]:
pip install langdetect

In [3]:
pip install twint

In [4]:
pip install tweet-preprocessor

In [5]:
  pip install geojson

In [6]:
import os
import json
import numpy as np
import pandas as pd
from preprocessor.api import clean, tokenize, parse
from nltk.stem import WordNetLemmatizer 
wnl = WordNetLemmatizer()
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string
import pickle
import regex as re
import string
import twint
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [13]:
import geopy.distance

coords_1 = [52.2296756, 21.0122287]
coords_2 = (52.406374, 16.9251681)
geopy.distance.vincenty(coords_1, coords_2).km

279.35290160386563

In [11]:
import geojson

path_to_json = 'data/geojson'
filen_list = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.geojson')]
 
keywords_list=["covid", 
"coronavirus", 
"covid19",
"covid19india",
"corona", 
"StaySafeStayHome",
"Stayhome",
"quarantine",
"INoLongerRemember",
"lockdown",
"COVIDIOTS",
"WFH",
"WorkingFromHome",
"FlattenTheCurve",
"GoCoronaGo",
"QuarantineAndChill",
"MyPandemicSurvivalPlan",
"CoroMustGo"]
hashtags_list = ["#"+w for w in keywords_list]

DIST = "20km"
SINCE = "2020-06-12"
UNTIL = "2020-06-13"
someattr="AC_NAME"

index_ward=[]
final=[]

keywords_list = keywords_list[:2]

for cityn in filen_list:
  with open(path_to_json+"/"+cityn,"rb") as f:
    city = geojson.load(f)
  index_ward_ = []
  for ward in city['features']:
    for kw in keywords_list:
      c = twint.Config()
      c.Search = kw
      ccor = np.array(ward['geometry']['coordinates'][0]).mean(axis=0)
      maxr=-1
      minr=1e10
      for p in ward['geometry']['coordinates'][0]:
        if geopy.distance.vincenty(p, list(cor)).km>maxr:
          maxr = geopy.distance.vincenty(p, cor).km
        if geopy.distance.vincenty(p, list(cor)).km<minr:
          minr = geopy.distance.vincenty(p, cor).km
      #c.Geo = str(ccor[0])+","+str(ccor[1])+","+DIST
      c.Geo = str(ccor[0])+","+str(ccor[1])+","+str((maxr+minr)/2)
      c.Lang="en"
      c.Since = SINCE
      c.Until = UNTIL
      c.Near = cityn[:-8].lower()
      c.Store_object = True
      c.Output = "t1.json"
      c.Hide_output = True
      c.Store_json = True
      c.Limit=20
      twint.run.Search(c)
      index_ward_.append(ward["properties"][someattr])
      tweets = []
      for i, line in enumerate(open('t1.json', 'r')):
          temp = json.loads(line)
          if len([w for w in temp["hashtags"] if w in hashtags_list])>0:
            tweets.append(temp)
      final+=tweets
  index_ward.append(index_ward_)

KeyboardInterrupt: ignored